# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [4]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
input_matches = "DNA Genealogy - Surnames_dad.csv"

multy = 1000

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [30]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.sqlite3')

In [5]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/{input_matches}')
matches_df

,surname,name,cm,url
0,Стемковський,Владислав Стемковський,113.2,https://www.myheritage.com.ua/dna/match/D-303A...
1,Стемкповський,Владислав Стемковський,113.2,https://www.myheritage.com.ua/dna/match/D-303A...
2,Дячук,Інна Сидорук,69.3,https://www.myheritage.com.ua/dna/match/D-303A...
3,Кавюк,Інна Сидорук,69.3,https://www.myheritage.com.ua/dna/match/D-303A...
4,Барчук,Інна Сидорук,69.3,https://www.myheritage.com.ua/dna/match/D-303A...
...,...,...,...,...
103,Лесик,Mikhail Furdik,26.3,https://www.myheritage.com.ua/dna/match/D-303A...
104,Маковський,Mikhail Furdik,26.3,https://www.myheritage.com.ua/dna/match/D-303A...
105,Осійчук,Mikhail Furdik,26.3,https://www.myheritage.com.ua/dna/match/D-303A...
106,Пасік,Mikhail Furdik,26.3,https://www.myheritage.com.ua/dna/match/D-303A...


# Підготовка таблиць

In [31]:
table_distribution = "distribution"
table_raw = "raw"

cursor = db.cursor()


In [38]:
cursor.execute(f'DROP TABLE IF EXISTS {table_distribution}')
cursor.execute(
    f'''
    CREATE TABLE {table_distribution} (
    	"region"	INTEGER NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"score"	REAL NOT NULL,
    	PRIMARY KEY("region","county","city")
    )
    '''
)

In [39]:
cursor.execute(f'DROP TABLE IF EXISTS {table_raw}')
cursor.execute(
    f'''
    CREATE TABLE "{table_raw}" (
    	"surname"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"region"	INTEGER NOT NULL,
    	"cnt"	INTEGER NOT NULL,
    	"match"	INTEGER NOT NULL,
    	"url"	TEXT NOT NULL
    );
    '''
)

In [40]:
db.commit()

# Групування збіженців

In [17]:
match_grouped = matches_df.groupby(["url", "cm", "name"]).surname

In [41]:
for (url, cm, match), surnames in match_grouped:
    print(match)
    cm_float = float(cm)
    str_surnames = "','".join(map(lambda t: t.upper(), surnames))
    select = f'''
        SELECT 
            COUNT(*) AS cn,
        	"Областьнародження" as region,
        	IFNULL("Районнародження",0) as county,
        	"Названаселеногопунктународження(Україна)" as city,
            "Прізвищє" as surname
        FROM people 
        where 
        	"Прізвищє" in ('{str_surnames}') 
        	AND "Стать"=1 
        	AND "Названаселеногопунктународження(Україна)" IS NOT NULL 
        	AND "Областьнародження" IS NOT NULL
        GROUP BY region,
        	county,
        	city,
            surname
    '''

    def rec(row):
        (cnt, region, county, city, surname) = row
        return {
            'surname': surname,
            "region": region,
            "county": county,
            "city": city,
            "cnt": cnt
        }

    cursor.execute(select)
    res = list(map(rec, cursor.fetchall()))
    total = 0
    for t in res:
        # print(t)
        total = total + t["cnt"]

    def update_params(r):
        score = (cm_float*r["cnt"]*multy)/total
        return (r["region"], r["county"], r["city"], score, score)

    def insert_raw_params(r):
        return (r["surname"], r["city"], r["county"], r["region"], r["cnt"], match, url)

    cursor.executemany(
        f'''
        INSERT INTO {table_distribution}(region, county, city, score)
          VALUES(?, ?, ?, ?)
          ON CONFLICT(region, county, city) DO UPDATE SET
            score=score+?;
        ''',
        list(map(update_params, res))
    )

    cursor.executemany(
        f'''
        INSERT INTO {table_raw}(surname, city, county, region, cnt, match, url)
          VALUES(?, ?, ?, ?, ?, ?, ?)
        ''',
        list(map(insert_raw_params, res))
    )
    
    db.commit()
    # break

print("All records processed")
    

Елена Алексеевна Петрукович
Mark Kiessling
Наталія Ковальчук
Mikhail Furdik
Yana Boug
Андрей Андриевский
Владислав Стемковський
Svitlana Slobodyanyuk
Roman Parfeniouk
Antonina Heitz
Pavel Ruban
Zhanna Khustochka
stanislav Ninaltovskyy
Андрій Коваленко
Інна Сидорук
Maksim Chykun
Janet Deyneka
Андрій Скиба
Василь Гребенюк
Роман Войцехівський
Viktoriya Шведченко
Natalia Etten
Diana Polianska
Oksana Vecherynska
Володимир Подзізей
Zhanna Klymenko
Юлія Рудюк
Nikolas 14
Paul Dmitruk
Ustynia Filimoniuk
Viacheslav Serdiuk
Tatiana Dolenko
Iryna Byshenko
All records processed


In [29]:
db.close()